# Two-Stages Deep Learning Project

<p>This project has been developped by the Montreal Hacknight community. It derives from the braindecode framework and is aimed at being integrable into the Moabb project.</p>

<p><ul>https://github.com/robintibor/braindecode</ul>
<ul>https://github.com/NeuroTechX/moabb</ul></p>

<p>This project wouldn't have been possible without the contribution of these people: Arna (...), Justin (...), Yannick Roy, Eamon Egan, (...) and it is currently being supported by Fred Simard (fs@re-ak.com)</p>

<p>This project consist in deriving a two-stages training process from the brain decode implementation of a Shallow Convolutional Network. The core motivation is to combine the strenghts of deep learning - that is being one of the most powerful ML algorithms out-there - while grinding down it's fall back - it requires an immense of data.</p>

<p>The goal is to develop a method to train the network in two stages:</p>
<p><ul>1) Train the network over a large dataset, this training forms some kind of prior over the network</ul>
<ul>2) Train the network a second time, this time on the dataset of interest</ul>
</p>

<p>The premise motivating this approach is that the second training is likely to proceed faster than what is normally required to train a deep net, without sacrificing the performance; and, we suspect that the second training will be able to exploit the specificities of the dataset of interest in a way to surpasses the first training, which was trained on a generalistic dataset.</p>

In [1]:
"""
#
# Download the github, which contains the package with the networks
#
"""

# reset to default base path
import os
os.chdir('/content/')

# if dl-eeg-playground already here, erase
!rm -rf dl-eeg-playground

# clone dl-eeg-playground and cd to it
!git clone https://github.com/NeuroTechX/dl-eeg-playground.git
os.chdir('dl-eeg-playground/brainDecode/towardMoabbIntegration')


Cloning into 'dl-eeg-playground'...
remote: Enumerating objects: 261, done.
remote: Total 261 (delta 0), reused 0 (delta 0), pack-reused 261
Receiving objects: 100% (261/261), 1.95 MiB | 1.75 MiB/s, done.
Resolving deltas: 100% (128/128), done.


In [2]:
#
# Download the whole BNCI 002-2014 dataset
#
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S01T.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S01E.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S02T.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S02E.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S03T.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S03E.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S04T.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S04E.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S05T.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S05E.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S06T.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S06E.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S07T.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S07E.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S08T.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S08E.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S09T.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S09E.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S10T.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S10E.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S11T.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S11E.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S12T.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S12E.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S13T.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S13E.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S14T.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S14E.mat

# move files into a dedicated folder
!mkdir BBCIData
!mv *.mat BBCIData

#install brain decode
!pip install braindecode -q

# install pytorch
# ref: http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

#!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.0-{platform}-linux_x86_64.whl torchvision
#  cu91/torch-0.4.0-cp36-cp36m-linux_x86_64.whl
import torch

import scipy.io as sio
import numpy as np
from os import listdir
from os.path import isfile, join

# prepare data containers
y = []
X = []

"""
Now, let's load data.

We read the file for the desired subject, and parse the data to extract:
- samplingRate
- trialLength
- X, a M x N x K matrix, which stands for trial x chan x samples
    - the actual values are 160 x 15 x 2560
- y, a M vector containing the labels {0,1}

ref: Dataset description: https://lampx.tugraz.at/~bci/database/002-2014/description.pdf
"""

folder = "BBCIData"

for f in listdir(folder):
    # read file
    d1T = sio.loadmat(folder + "/" + f)
    
    samplingRate = d1T['data'][0][0][0][0][3][0][0]
    trialLength = 7*samplingRate

    # run through all training runs
    for run in range(len(d1T['data'][0])):
        y.append(d1T['data'][0][run][0][0][2][0]) # labels
        timestamps = d1T['data'][0][run][0][0][1][0] # timestamps
        rawData = d1T['data'][0][run][0][0][0].transpose() # chan x data

        # parse out data based on timestamps
        for start in timestamps:
            end = start + trialLength
            X.append(rawData[:,start:end]) #15 x 2560

    del rawData
    del d1T

# arrange data into num7py arrays
# also torch expect float32 for samples
# and int64 for labels {0,1}
X = np.array(X).astype(np.float32)
y = (np.array(y).flatten()-1).astype(np.int64)
print()
print(X.shape)
print(y.shape)


# rand permute dataset
idx = np.random.permutation(X.shape[0])

X = X[idx,:,:]
y = y[idx]

--2018-10-25 22:48:57--  http://bnci-horizon-2020.eu/database/data-sets/002-2014/S01T.mat
Resolving bnci-horizon-2020.eu (bnci-horizon-2020.eu)... 91.227.204.35
Connecting to bnci-horizon-2020.eu (bnci-horizon-2020.eu)|91.227.204.35|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://lampx.tugraz.at/~bci/database/002-2014/S01T.mat [following]
--2018-10-25 22:48:58--  https://lampx.tugraz.at/~bci/database/002-2014/S01T.mat
Resolving lampx.tugraz.at (lampx.tugraz.at)... 129.27.124.207
Connecting to lampx.tugraz.at (lampx.tugraz.at)|129.27.124.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 39794870 (38M)
Saving to: ‘S01T.mat’

S01T.mat            100%[===================>]  37.95M  6.82MB/s    in 11s     

2018-10-25 22:49:11 (3.34 MB/s) - ‘S01T.mat’ saved [39794870/39794870]

--2018-10-25 22:49:12--  http://bnci-horizon-2020.eu/database/data-sets/002-2014/S01E.mat
Resolving bnci-horizon-2020.eu (bnci-horizon-2020.eu)... 91.

In [22]:

# load the general network Trainer
from brainDecodeSKLearnWrapper.ShallowFBCSPNet_GeneralTrainer import ShallowFBCSPNet_GeneralTrainer

classifier = ShallowFBCSPNet_GeneralTrainer(nb_epoch=25)
classifier.fit(X,y)


Epoch 0
Train  Loss: 0.81426
Train  Accuracy: 50.8%
Test   Loss: 0.89167
Test   Accuracy: 46.8%
Epoch 1
Train  Loss: 1.05222
Train  Accuracy: 49.9%
Test   Loss: 1.06232
Test   Accuracy: 49.6%
Epoch 2
Train  Loss: 0.91004
Train  Accuracy: 51.6%
Test   Loss: 0.97330
Test   Accuracy: 48.2%
Epoch 3
Train  Loss: 0.91777
Train  Accuracy: 50.4%
Test   Loss: 0.95551
Test   Accuracy: 48.9%
Epoch 4
Train  Loss: 0.71086
Train  Accuracy: 54.3%
Test   Loss: 0.74631
Test   Accuracy: 51.4%
Epoch 5
Train  Loss: 0.70673
Train  Accuracy: 55.2%
Test   Loss: 0.74628
Test   Accuracy: 48.9%
Epoch 6
Train  Loss: 0.66773
Train  Accuracy: 58.5%
Test   Loss: 0.70476
Test   Accuracy: 50.7%
Epoch 7
Train  Loss: 0.66556
Train  Accuracy: 59.1%
Test   Loss: 0.70835
Test   Accuracy: 50.4%
Epoch 8
Train  Loss: 0.65287
Train  Accuracy: 61.9%
Test   Loss: 0.69702
Test   Accuracy: 53.9%
Epoch 9
Train  Loss: 0.65098
Train  Accuracy: 60.8%
Test   Loss: 0.67780
Test   Accuracy: 56.8%
Epoch 10
Train  Loss: 0.64131
Train  Acc

ShallowFBCSPNet_GeneralTrainer(filter_time_length=75, n_filters_spat=5,
                n_filters_time=10, nb_epoch=25, pool_time_length=60,
                pool_time_stride=30)

In [0]:
# TODO: Save network to file

In [31]:

#from brainDecodeSKLearnWrapper.ShallowFBCSPNet_SpecializedTrainer import ShallowFBCSPNet_SpecializedTrainer


!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S01T.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S01E.mat
!mkdir BBCIData
!mv *.mat BBCIData


"""
Load a single user to specialize the training
"""

folder = "BBCIData"
datasetID = 1

# prepare data containers
y_single = []
X_single = []

f = listdir(folder)[datasetID]
  
# read file
d1T = sio.loadmat(folder + "/" + f)

samplingRate = d1T['data'][0][0][0][0][3][0][0]
trialLength = 7*samplingRate

# run through all training runs
for run in range(len(d1T['data'][0])):
    y_single.append(d1T['data'][0][run][0][0][2][0]) # labels
    timestamps = d1T['data'][0][run][0][0][1][0] # timestamps
    rawData = d1T['data'][0][run][0][0][0].transpose() # chan x data

    # parse out data based on timestamps
    for start in timestamps:
        end = start + trialLength
        X_single.append(rawData[:,start:end]) #15 x 2560

del rawData
del d1T

# arrange data into num7py arrays
# also torch expect float32 for samples
# and int64 for labels {0,1}
X_single = np.array(X_single).astype(np.float32)
y_single = (np.array(y_single).flatten()-1).astype(np.int64)
print()
print(X_single.shape)
print(y_single.shape)


# rand permute dataset
idx = np.random.permutation(X_single.shape[0])

X_single = X_single[idx,:,:]
y_single = y_single[idx]






--2018-10-26 01:08:50--  http://bnci-horizon-2020.eu/database/data-sets/002-2014/S01T.mat
Resolving bnci-horizon-2020.eu (bnci-horizon-2020.eu)... 91.227.204.35
Connecting to bnci-horizon-2020.eu (bnci-horizon-2020.eu)|91.227.204.35|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://lampx.tugraz.at/~bci/database/002-2014/S01T.mat [following]
--2018-10-26 01:08:51--  https://lampx.tugraz.at/~bci/database/002-2014/S01T.mat
Resolving lampx.tugraz.at (lampx.tugraz.at)... 129.27.124.207
Connecting to lampx.tugraz.at (lampx.tugraz.at)|129.27.124.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 39794870 (38M)
Saving to: ‘S01T.mat’

S01T.mat            100%[===================>]  37.95M  6.81MB/s    in 10s     

2018-10-26 01:09:02 (3.68 MB/s) - ‘S01T.mat’ saved [39794870/39794870]

--2018-10-26 01:09:04--  http://bnci-horizon-2020.eu/database/data-sets/002-2014/S01E.mat
Resolving bnci-horizon-2020.eu (bnci-horizon-2020.eu)... 91.

In [36]:
# TODO: Load network to file
from brainDecodeSKLearnWrapper.ShallowFBCSPNet_SpecializedTrainer import ShallowFBCSPNet_SpecializedTrainer


#Xhere = X  #full dataset
#yhere = y
Xhere = X_single  #single user
yhere = y_single

# rand permute dataset
idx = np.random.permutation(X_single.shape[0])

Xhere = Xhere[idx,:,:]
Xhere = Xhere[idx]

specializedClassifier = None
specializedClassifier = ShallowFBCSPNet_SpecializedTrainer(network=classifier.model) # this will effectively save the network


specializedClassifier = ShallowFBCSPNet_SpecializedTrainer(filename="myModel.pth")
specializedClassifier.model.cuda() # patch

#specializedClassifier._loadFromFile("myModel.pth")
#specializedClassifier = ShallowFBCSPNet_SpecializedTrainer(network=classifier.model)
specializedClassifier.lr = 0.001
#specializedClassifier.configure(initial_lr=0.01)
specializedClassifier.fit(Xhere,yhere)


Epoch 0
Train  Loss: 0.74990
Train  Accuracy: 55.8%
Test   Loss: 0.91977
Test   Accuracy: 37.5%
Epoch 1
Train  Loss: 0.68242
Train  Accuracy: 55.8%
Test   Loss: 0.84993
Test   Accuracy: 37.5%
Epoch 2
Train  Loss: 0.62654
Train  Accuracy: 57.7%
Test   Loss: 0.80075
Test   Accuracy: 37.5%
Epoch 3
Train  Loss: 0.58176
Train  Accuracy: 71.2%
Test   Loss: 0.76497
Test   Accuracy: 50.0%
Epoch 4
Train  Loss: 0.54284
Train  Accuracy: 78.8%
Test   Loss: 0.72861
Test   Accuracy: 50.0%
Epoch 5
Train  Loss: 0.51042
Train  Accuracy: 80.8%
Test   Loss: 0.70225
Test   Accuracy: 50.0%
Epoch 6
Train  Loss: 0.48177
Train  Accuracy: 82.7%
Test   Loss: 0.68461
Test   Accuracy: 62.5%
Epoch 7
Train  Loss: 0.45576
Train  Accuracy: 82.7%
Test   Loss: 0.66767
Test   Accuracy: 62.5%
Epoch 8
Train  Loss: 0.43231
Train  Accuracy: 84.6%
Test   Loss: 0.66055
Test   Accuracy: 62.5%
Epoch 9
Train  Loss: 0.41055
Train  Accuracy: 88.5%
Test   Loss: 0.65576
Test   Accuracy: 62.5%
Epoch 10
Train  Loss: 0.39075
Train  Acc

KeyboardInterrupt: ignored

In [18]:
!git pull


remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 6 (delta 5), reused 6 (delta 5), pack-reused 0
Unpacking objects: 100% (6/6), done.
From https://github.com/NeuroTechX/dl-eeg-playground
   cf81c2b..0286a3d  master     -> origin/master
Updating cf81c2b..0286a3d
Fast-forward
 .../brainDecodeSKLearnWrapper/ShallowFBCSPNet_SpecializedTrainer.py      | 1 +
 1 file changed, 1 insertion(+)


Best scores so far:

Subject 0: 76.9%
Subject 1 50.0%


